# Infer and Compare Result on different Validatio Dataset( Set5, Set14)

* we will make final function to predict images using different model and different validation dataset

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
os.chdir('/content/drive/MyDrive/CS2')

In [ ]:
!pip install silence-tensorflow

In [163]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [ ]:
set5 = sorted(os.listdir('SET_5/Set5/'))
print('Following are different Downgrading options ')
print('\n')
set5

Following are different Downgrading options 




['GTmod12', 'LRbicx2', 'LRbicx3', 'LRbicx4', 'original']

In [130]:
# create dataframe of low and high resolution data from set5 
SET_5_DIR = 'SET_5/Set5/'

def Get_set_5_14_paths(data_dir,downgrading_option):
    # list downgrading options
    options = sorted(os.listdir(data_dir))

    # condition for different downgrading options
    if downgrading_option == 'GTmod12':
        set5 = sorted(os.listdir(data_dir + 'GTmod12'))
    elif downgrading_option == 'LRbicx2':
        set5 = sorted(os.listdir(data_dir + 'LRbicx2'))
    elif downgrading_option == 'LRbicx3':
        set5 = sorted(os.listdir(data_dir + 'LRbicx3'))
    elif downgrading_option == 'LRbicx4':
        set5 = sorted(os.listdir(data_dir + 'LRbicx4'))
    else:
        raise Exception('Plese select downgrading options from {}'.format(options[:-1]))
    
    # Original images 
    original = sorted(os.listdir(data_dir + 'original'))
    original_path = [os.path.join(data_dir,'original',i) for i in original]

    set5_path = [os.path.join(data_dir,downgrading_option,i) for i in set5]

    #df = pd.DataFrame(data = {'Original' : original_path, downgrading_option : set5_path})

    return original_path,set5_path


In [157]:
original_path,set5_path = Get_set_5_14_paths(SET_5_DIR,'LRbicx4')

In [158]:
original_path

['SET_5/Set5/original/baby.png',
 'SET_5/Set5/original/bird.png',
 'SET_5/Set5/original/butterfly.png',
 'SET_5/Set5/original/head.png',
 'SET_5/Set5/original/woman.png']

In [159]:
set5_path

['SET_5/Set5/LRbicx4/baby.png',
 'SET_5/Set5/LRbicx4/bird.png',
 'SET_5/Set5/LRbicx4/butterfly.png',
 'SET_5/Set5/LRbicx4/head.png',
 'SET_5/Set5/LRbicx4/woman.png']

## Helper Function to load and plot images

In [152]:
from PIL import Image

def load_image(path):
    return np.array(Image.open(path))

def plot_images(lr,sr,figsize = (20,10)):
    fig,axes = plt.subplots(nrows = 1, ncols = 2, figsize=figsize)
    for ax in axes.flatten():
        ax.axis('off')
    axes[0].set_title('Low resolution',fontdict={'fontsize': 15,'fontweight': 'bold'})
    axes[0].imshow(lr)
    axes[1].set_title('Predicted(SR)',fontdict={'fontsize': 15,'fontweight': 'bold'})
    axes[1].imshow(sr)  

def plot_samples(lr,sr,hr,figsize=(10,10)):
    plt.figure(figsize=figsize)

    images = [lr, sr, hr]
    titles = ['LR', f'SR (x{sr.shape[0] // lr.shape[0]})(predicted)', 'HR(Original)']

    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, 3, i+1)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])  


In [193]:
def get_sr_image(model, lr_img):
    '''' Get super resolution image from model'''
    # expand dims of lr image for prediction
    lr_img = tf.expand_dims(lr_img,axis=0)
    # convert dtype to float32
    lr_img = tf.cast(lr_img, tf.float32)

    # predict image
    sr_img = model(lr_img)
    sr_img = tf.clip_by_value(sr_img, 0, 255)
    sr_img = tf.round(sr_img)
    sr_img = tf.cast(sr_img, tf.uint8)
    return sr_img

# Load EDSR pretrained_model On Bicubic_X4

In [65]:
!ls

blur.jpeg	   edsr_unknown_x4.h5		 sumit_blur.jpg
blur.png	   final.ipynb			 test.png
blurred-image.png  Model			 wdsr_300000.h5
checkpoint	   postprocess_gaussianblur.jpg  WDSR_bicubic_x4.ipynb
CS2_EDA.ipynb	   preprocessing.ipynb		 wdsr_unknown_x4.h5
edsr_300000.h5	   real_world_blur		 WDSR_unknown_x4.ipynb
edsr_350000.h5	   SET_5


In [83]:
import models

In [155]:
edsr_model = models.EDSR(scale=4,res_blocks=16)
wdsr_b_model = models.WDSR_B(scale=4,num_res_blocks=32)

In [86]:
# load pretrained weighs for bicubic_x4
edsr_model.load_weights('edsr_350000.h5')

In [156]:
wdsr_b_model.load_weights('wdsr_300000.h5')

# SET_5 Bicubic_x4
### Predict Images from trained EDSR model on Bicubic_X4

In [122]:
for lr,hr in zip(set5_path,original_path):
    lr_img = load_image(lr)
    hr_img = load_image(hr)
    sr_img = get_sr_image(edsr_model,lr_img)
    plot_samples(lr_img,sr_img[0],hr_img,figsize=(20,10))

Output hidden; open in https://colab.research.google.com to view.

## Predict Images from trained WDSR_B model on Bicubic_X4

In [164]:
for lr,hr in zip(set5_path,original_path):
    lr_img = load_image(lr)
    hr_img = load_image(hr)
    sr_img = get_sr_image(wdsr_b_model,lr_img)
    plot_samples(lr_img,sr_img[0],hr_img,figsize=(20,10))

Output hidden; open in https://colab.research.google.com to view.

# SET_5 Bicubic_x2 using EDSR

In [165]:
original_path,set5_path = Get_set_5_14_paths(SET_5_DIR,'LRbicx2')

In [124]:
for lr,hr in zip(set5_path,original_path):
    lr_img = load_image(lr)
    hr_img = load_image(hr)
    sr_img = get_sr_image(edsr_model,lr_img)
    plot_samples(lr_img,sr_img[0],hr_img,figsize=(20,10))

Output hidden; open in https://colab.research.google.com to view.

# SET_14

In [125]:
os.makedirs('SET_14/',exist_ok=True)

In [ ]:
!unzip '/content/drive/MyDrive/Classical/Set14.zip' -d 'SET_14/'

In [131]:
SET_14_DIR = 'SET_14/Set14/'

## SET_14 Bicubic_X4 Using EDSR

In [219]:
original_path,set14_path = Get_set_5_14_paths(SET_14_DIR,'LRbicx4')

In [141]:
for lr,hr in zip(set14_path,original_path):
    lr_img = load_image(lr)
    hr_img = load_image(hr)
    sr_img = get_sr_image(edsr_model,lr_img)
    plot_samples(lr_img,sr_img[0],hr_img,figsize=(20,10))

Output hidden; open in https://colab.research.google.com to view.

## SET_14 Bicubic_X4 Using WDSR_B


In [169]:
for lr,hr in zip(set14_path,original_path):
    lr_img = load_image(lr)
    hr_img = load_image(hr)
    sr_img = get_sr_image(wdsr_b_model,lr_img)
    plot_samples(lr_img,sr_img[0],hr_img,figsize=(20,10))

Output hidden; open in https://colab.research.google.com to view.

In [228]:
def final_function(model,lr_image):
    '''' Get super resolution image from model'''
    # expand dims of lr image for prediction
    if len(lr_image.shape) == 3 :
        lr_image = tf.expand_dims(lr_image,axis=0)
     
    # convert dtype to float32
    lr_img = tf.cast(lr_image, tf.float32)

    # predict image
    sr_img = model(lr_img)
    sr_img = tf.clip_by_value(sr_img, 0, 255)
    sr_img = tf.round(sr_img)
    sr_img = tf.cast(sr_img, tf.uint8)
    return sr_img


# Conclusion :
*  As we can see EDSR and WDSR_B results. there is not too much difference in EDSR and WDSR_B results.
* Reason behind not getting much more difference is there is only difference 1% in validation PSNR of EDSR and WDSR
* For further improvement we can train SRGAN model